In [1]:
import cv2
import pandas as pd
import numpy as np
import time

In [2]:
def create_background(cap , num_frames=30) :
    print("Capturing Background, Please move out of the way!!!")
    backgrounds = []

    for i in range(num_frames):
        ret, frame = cap.read()
        if ret:
            backgrounds.append(frame)
        else:
            print(f"Warning: Could not read the frame {i+1}/{num_frames}")
        time.sleep(0.1)

    if backgrounds:
        return np.median(backgrounds, axis=0).astype(np.uint8)
    else:
        raise ValueError("Could not capture any frames for the background")

In [3]:
def create_mask(frame, lower_color, upper_color):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_color, upper_color)
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, np.ones((3,3), np.uint8), iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, np.ones((3,3), np.uint8), iterations=1)
    return mask

In [4]:
def apply_cloak_effect(frame, mask, background):
    mask_inv = cv2.bitwise_not(mask)

    # Convert 1-channel mask to 3-channel
    mask_3ch = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    mask_inv_3ch = cv2.cvtColor(mask_inv, cv2.COLOR_GRAY2BGR)

    fg = cv2.bitwise_and(frame, mask_inv_3ch)
    bg = cv2.bitwise_and(background, mask_3ch)
    return cv2.add(fg, bg)

In [5]:
def main():
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error opening the camera")
        return
    
    try:
        background  = create_background(cap)
    except ValueError as e:
        print(f"Error: {e}")
        cap.release()
        return
    
    lower_red = np.array([0, 120, 70])
    upper_red = np.array([10, 255, 255])

    print("Starting main loop")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Could not read frame")
            time.sleep(1)
            return
        
        mask = create_mask(frame, lower_red, upper_red)
        result = apply_cloak_effect(frame, mask, background)

        cv2.imshow("Invisibilty Cloak", result)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [6]:
main()

Capturing Background, Please move out of the way!!!
Starting main loop


KeyboardInterrupt: 

In [ ]:
pkill python

SyntaxError: invalid syntax (2150071729.py, line 1)